# WGAN version v3 training + testing all in one file

## only schizophrenia, but pick only 50 random for training

In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split

2024-01-31 01:04:04.152576: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 01:04:04.188853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 01:04:04.188884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 01:04:04.189864: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 01:04:04.196963: I tensorflow/core/platform/cpu_feature_guar

In [12]:
# List all physical devices of type 'GPU'
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    print(f'Number of GPUs available: {len(gpus)}')
    for i, gpu in enumerate(gpus):
        print(f'GPU {i}: {gpu}')
else:
    print('No GPU detected.')

Number of GPUs available: 1
GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [13]:
# full list of all schizophrenia IDs from csv, 86 in total but not all satisfy have t>90

full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
'''met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624'] '''

met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
   ]

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

'''met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]'''

met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
]

In [14]:
def build_wgan_generator(z_dim):
    # Noise input
    z_input = Input(shape=(z_dim,))

    # Generator network
    model = Sequential()
    
    # Start with a Dense layer to an initial shape that's smaller than the final target
    model.add(Dense(128 * 7 * 7 * 9, input_dim=z_dim))  # Adjust to match an initial volume
    model.add(LeakyReLU(alpha=0.01))
    model.add(Reshape((7, 7, 9, 128)))  # This is the initial volume
    
    # Begin upsampling to the desired size
    model.add(Conv3DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    
    # Continue upsampling
    model.add(Conv3DTranspose(32, kernel_size=3, strides=(3, 3, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    
    # Final upsampling step to reach just under the target size
    model.add(Conv3DTranspose(1, kernel_size=3, strides=(2, 2, 2), padding='same', activation='tanh'))

    # Output tensor
    output = model(z_input)

    return Model(z_input, output)

In [15]:
def build_wgan_critic(img_shape):
    # Image input
    img_input = Input(shape=img_shape)

    # Critic network
    x = Conv3D(64, kernel_size=3, strides=2, padding='same')(img_input)
    x = LeakyReLU(alpha=0.01)(x)

    x = Conv3D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.01)(x)
    
    x = Flatten()(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.01)(x)
    
    # Output a score for realness (no sigmoid activation)
    output = Dense(1)(x)

    return Model(img_input, output)

In [16]:
def gradient_penalty(real_images, fake_images, critic):
    batch_size = tf.shape(real_images)[0]

    # Alpha for interpolation - shape: (batch_size, 1, 1, 1, 1)
    alpha = tf.random.uniform([batch_size, 1, 1, 1, 1], 0., 1.)

    # Interpolated images - shape: (batch_size, 84, 84, 72, 1)
    interpolated_images = (real_images * alpha) + (fake_images * (1 - alpha))

    with tf.GradientTape() as tape:
        tape.watch(interpolated_images)
        # Critic now only takes the images as input
        predictions = critic(interpolated_images, training=True)

    # Calculate the gradients with respect to the interpolated images
    gradients = tape.gradient(predictions, [interpolated_images])[0]

    # Compute the norm of the gradients - reduce over all dimensions except the batch dimension
    gradients_norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3, 4]))

    # Penalize the gradient norm deviation from 1
    gp = tf.reduce_mean((gradients_norm - 1.) ** 2)
    return gp

In [17]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0]/image.shape[0],
        new_shape[1]/image.shape[1],
        new_shape[2]/image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation

In [18]:
def train_wgan_gp(generator, critic, dataset, epochs, z_dim, lambda_gp, critic_optimizer, generator_optimizer):
    # Lists to keep track of losses
    critic_losses = []
    generator_losses = []

    # Directory for saving checkpoints
    checkpoint_dir = "wgan_gp_checkpoints"
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    for epoch in range(epochs):
        epoch_critic_loss = 0.0
        epoch_generator_loss = 0.0
        num_batches = 0
        
        for batch in dataset:
            # Assuming dataset yields only images
            real_imgs = batch[0]

            num_batches += 1
            batch_size = real_imgs.shape[0]

            # Resize real images to match the expected dimensions of the critic
            real_imgs_resized = np.array([resize_image(img, (84, 84, 72)) for img in real_imgs])
            real_imgs_resized = np.expand_dims(real_imgs_resized, axis=-1)  # Add channel dimension

            # Train the critic
            for _ in range(5):  # Critic is often trained more frequently
                with tf.GradientTape() as tape:
                    # Generate fake images
                    z = tf.random.normal([batch_size, z_dim])
                    fake_imgs = generator(z, training=True)

                    # Get critic scores for real and fake images
                    real_output = critic(real_imgs_resized, training=True)
                    fake_output = critic(fake_imgs, training=True)

                    # Calculate critic loss
                    critic_cost = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)
                    gp = gradient_penalty(real_imgs_resized, fake_imgs, critic)
                    critic_loss = critic_cost + lambda_gp * gp

                # Update critic weights
                critic_grads = tape.gradient(critic_loss, critic.trainable_variables)
                critic_optimizer.apply_gradients(zip(critic_grads, critic.trainable_variables))
                epoch_critic_loss += critic_loss

            # Train the generator
            z = tf.random.normal([batch_size, z_dim])

            with tf.GradientTape() as tape:
                fake_imgs = generator(z, training=True)
                fake_output = critic(fake_imgs, training=True)
                generator_loss = -tf.reduce_mean(fake_output)

            # Update generator weights
            generator_grads = tape.gradient(generator_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(generator_grads, generator.trainable_variables))
            epoch_generator_loss += generator_loss

            # print the losses for each batch
            print(f'Epoch: {epoch}, Batch: {num_batches}, Critic Loss: {critic_loss}, Generator Loss: {generator_loss}')

        # Checkpointing every n epochs
        if (epoch + 1) % 2 == 0:
            generator.save_weights(os.path.join(checkpoint_dir, f"generator_epoch_{epoch+1}.h5"))
            critic.save_weights(os.path.join(checkpoint_dir, f"critic_epoch_{epoch+1}.h5"))
            print(f"Checkpoint: Saved model weights at epoch {epoch+1}")
            
        # Record the average losses for this epoch
        critic_losses.append(epoch_critic_loss / num_batches)
        generator_losses.append(epoch_generator_loss / num_batches)

    return critic_losses, generator_losses


In [19]:
def plot_losses(d_losses, g_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label='Discriminator Loss')
    plt.plot(g_losses, label='Generator Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


In [ ]:
random.seed(123)

In [24]:
total_accuracy = 0
total_loss = 0
num_iterations = 1

for i in range(num_iterations):

    ''' Choosing random training samples and store remaining 9 test'''
    train_ids = random.sample(met_requirement_schizophrenia_ids, 5)
    test_ids = [id for id in met_requirement_schizophrenia_ids if id not in train_ids]

    ''' File loading '''
    # Specify the directory and file pattern
    directory_path = '../4D'
    file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

    # Construct the full path pattern
    path_pattern = f'{directory_path}/{file_pattern}'

    # Use glob to find all matching files
    matching_files = glob.glob(path_pattern)

    # Initialize lists to store the processed image data, corresponding labels, and filenames
    image_data = []
    labels = []  # 1 for schizophrenia, 0 for non-schizophrenia
    schizophrenia_files = []

    # Lists for files with insufficient time dimensions
    insufficient_time_files = []
    insufficient_time_ids = []

    # Counters for each category
    schizophrenia_count = 0
    processed_files_count = 0

    # Loop through the matching files
    for file_path in matching_files:
        # Extract the filename
        filename = os.path.basename(file_path)
        
        # Extract the ID from the filename
        file_id = filename.split('_')[0]
        
        # Load the file
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        # Check if the time dimension is at least 90
        if t1_data.shape[3] < 90:
            insufficient_time_files.append(filename)
            insufficient_time_ids.append(file_id)
            continue  # Skip this file

        # Determine the label based on the ID and increment counters
        if file_id in train_ids:
            label = 1  # Schizophrenia
            schizophrenia_count += 1
            schizophrenia_files.append(filename)
        else:
            continue  # Skip files with IDs not in the provided lists
        
        # Collapse one of the axes by summing
        t1_data_collapsed = np.sum(t1_data, axis=1)
        
        # Append the collapsed image data and label to the respective lists
        image_data.append(t1_data_collapsed)
        labels.append(label)

        # Increment the counter
        processed_files_count += 1

    # Print the total number of files processed for each category and their filenames
    print(f"Total number of files successfully processed: {processed_files_count}")
    print(f"Total number of schizophrenia files: {schizophrenia_count}")
    print("Schizophrenia files:", schizophrenia_files)

    '''Determine the maximum z-dimension size '''
    max_z_size = max(img.shape[2] for img in image_data)

    ''' normalization '''
    image_data_normalized = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in image_data]

    ''' padding of images data '''
    # Pad each image to have a consistent z-dimension size
    padded_data = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size - img.shape[2])), mode='constant') for img in image_data_normalized]

    # Now convert the padded data list to a numpy array
    padded_data_array = np.array(padded_data)

    ''' loading the data for training '''
    train_images = padded_data_array
    # Define batch size
    batch_size = 10
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, labels)).shuffle(len(train_images)).batch(batch_size)

    
    ''' setting up parameters'''
    # Image shape and other parameters
    img_shape = (84, 84, 72, 1)
    z_dim = 100

    # Create the generator and critic
    generator = build_wgan_generator(z_dim)
    critic = build_wgan_critic(img_shape)

    critic_optimizer = Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.999)
    generator_optimizer = Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.999)

    epochs = 10
    lambda_gp = 10  # Gradient penalty coefficient

    '''Train the WGAN-GP '''

    critic_losses, generator_losses = train_wgan_gp(
        generator, 
        critic, 
        train_dataset, 
        epochs, 
        z_dim,
        lambda_gp, 
        critic_optimizer, 
        generator_optimizer
    )
    
    ''' after training, retrieve the weights of the critic'''

    checkpoint_dir = '/wgan_gp_checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                    discriminator_optimizer=critic_optimizer,
                                    generator=generator,
                                    discriminator=critic)
    
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

    ''' transfer learning '''
    
    ''' adding classification layers'''
    new_model = Sequential()
    for i in range(len(critic.layers)-1):  # Excluding the last layer
        layer = critic.layers[i]
        layer.trainable = False  # Freeze the layer
        new_model.add(layer)

    # Adding additional layers
    new_model.add(Dropout(0.3))
    new_model.add(Dense(1, activation='sigmoid', name = 'dense_6'))  # Sigmoid for binary classification

    ''' Prepare data before training the classifer model '''
    # Resize each image in the padded data array
    resized_images = [resize_image(img, (84, 84, 72)) for img in padded_data]

    # Convert the resized data to a numpy array
    resized_images_array = np.array(resized_images)

    # Define batch size
    batch_size = 10

    # labels array corresponding to the images
    labels_array = np.array(labels)
    
    ''' train classifier normally like in transfer-learning v2'''
    # Split the data into training and evaluation sets (80% train, 20% eval)
    X_train, X_eval, y_train, y_eval = train_test_split(resized_images_array, labels_array, test_size=0.2, random_state=42)

    # Convert to TensorFlow datasets with labels
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
    eval_dataset = tf.data.Dataset.from_tensor_slices((X_eval, y_eval)).batch(batch_size)

    # Compile the model (if using a new instance of the model for each fold)
    new_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = new_model.fit(
        train_dataset,
        epochs=14,
        validation_data=eval_dataset
    )

    ''' make test set'''
    schizophrenia_test_ids = test_ids
    control_test_ids = random.sample(met_requirement_control_ids, 8)

    ''' load test images to make test set'''
    # Now proceed with loading and preprocessing the images for these IDs
    test_image_data = []
    test_labels = []
    # Load and preprocess the images
    test_image_data = []
    test_labels = []

    # Combine all test IDs
    test_ids = control_test_ids + schizophrenia_test_ids

    # Loop through the matching files and filter based on test IDs
    for file_path in matching_files:
        filename = os.path.basename(file_path)
        file_id = filename.split('_')[0]

        # Process only if the ID is in the test set
        if file_id in test_ids:
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()

            # Ensure sufficient time dimension
            if t1_data.shape[3] < 90:
                continue

            # Collapse one of the axes by summing
            t1_data_collapsed = np.sum(t1_data, axis=1)

            # Resize, normalize, and add dimension as done in the training data preparation
            processed_image = resize_image(t1_data_collapsed, (84, 84, 72))
            processed_image_normalized = (processed_image - np.min(processed_image)) / (np.max(processed_image) - np.min(processed_image)) * 2 - 1
            processed_image_final = np.expand_dims(processed_image_normalized, axis=-1)

            test_image_data.append(processed_image_final)
            label = 1 if file_id in schizophrenia_test_ids else 0
            test_labels.append(label)

    # Convert to numpy arrays
    test_images_array = np.array(test_image_data)
    test_labels_array = np.array(test_labels)


    test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)
    test_loss = 0
    test_accuracy = 0
    num_batches = 0
    actual_labels = []
    predicted_labels_list = []
    # Manually iterate over the test dataset
    for images, labels in test_dataset:
        # Make predictions
        predictions = new_model.predict(images)

        # Calculate loss for the batch
        loss = tf.keras.losses.binary_crossentropy(labels, predictions)
        test_loss += tf.reduce_mean(loss).numpy()

        # Process predictions
        predicted_labels_batch = tf.cast(tf.round(predictions), dtype=tf.int64)
        predicted_labels_list.extend(predicted_labels_batch.numpy().flatten())
        actual_labels.extend(labels.numpy().flatten())

        # Calculate accuracy for the batch
        accuracy = tf.reduce_mean(tf.cast(tf.equal(labels, predicted_labels_batch), dtype=tf.float32))
        test_accuracy += accuracy.numpy()

        num_batches += 1

    # Calculate average loss and accuracy
    total_loss += test_loss
    total_accuracy += test_accuracy
    
    # Print test results
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    print("Actual labels vs. Predicted labels:")
    for i in range(len(actual_labels)):
        print(f"Image {i+1}: Actual: {actual_labels[i]}, Predicted: {predicted_labels_list[i]}")

# Calculate and print the average loss and accuracy over all iterations
average_test_loss = total_loss / num_iterations
average_test_accuracy = total_accuracy / num_iterations
print(f"\nAverage Test Loss over {num_iterations} iterations: {average_test_loss}")
print(f"Average Test Accuracy over {num_iterations} iterations: {average_test_accuracy}")

Total number of files successfully processed: 5
Total number of schizophrenia files: 5
Schizophrenia files: ['A00000456_0013_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz', 'A00001452_0014_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz', 'A00001251_0015_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz', 'A00004507_0014_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz', 'A00000838_0013_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz']
Epoch: 0, Batch: 1, Critic Loss: -20.439830780029297, Generator Loss: -0.33459481596946716
Epoch: 1, Batch: 1, Critic Loss: -120.14432525634766, Generator Loss: -1.4736920595169067
Checkpoint: Saved model weights at epoch 2
Epoch: 2, Batch: 1, Critic Loss: -242.5237274169922, Generator Loss: -2.842027187347412
Epoch: 3, Batch: 1, Critic Loss: -321.0494384765625, Generator Loss: -3.6108181476593018
Checkpoint: Saved model weights at epoch 4
Epoch: 4, Batch: 1, Critic Loss: -363.6291198730469, Generator Loss: -4.629116535186768

In [ ]:
'''
# Generate a single fake image
z = np.random.normal(0, 1, (1, z_dim))
generated_image = generator.predict(z)[0]  # [0] to get the single image from the batch

# Get a single real image from the dataset
real_images = next(iter(train_dataset))[0]  # Assuming the dataset yields only images

# Take the first real image from the batch for comparison
real_image = real_images[0]

# Plot the real and fake images side by side
plt.figure(figsize=(10, 5))

# Plot real image
plt.subplot(1, 2, 1)
plt.imshow(real_image[:, :, 10, 0])  # Adjust indexing and color map as needed
plt.title('Real Image')
plt.axis('off')

# Plot fake image
plt.subplot(1, 2, 2)
plt.imshow(generated_image[:, :, 10, 0])  # Adjust indexing and color map as needed
plt.title('Generated Image')
plt.axis('off')

plt.tight_layout()
plt.show()
'''

In [ ]:
#generator.save('wgan_generator_model_v3.h5')
#critic.save('wgan_critic_model_v3.h5')